# Inverting evolution for Ising model
The goal is to obtain the inverse evolution: $e^{-itH}\to e^{itH}$

In [1]:
from simple_exact_diagonalization_routines.local_matrix_class import *
from scipy.linalg import expm
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
from class_ising_model import *
import itertools
from class_double_bracket_flow import *
import math

## Nearest-neighbor

### Test 1: 1D Nearest neighbor
First, we observe that for a nearest-neighbor model $H_{NN} = \sum_i Z_i Z_{i+1}$, $X_{odd} = \prod_i X_{2i-1}$, satisfies:

   \begin{align}
        X_\text{odd} H_\text{NN}X_\text{odd} =-H_\text{NN}
    \end{align}

In the 4 qubit case:

$H_{NN} = Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4$

$H_{odd} = X_1X_3$

$H_{odd}H_{NN}H_{odd} = -Z_1Z_2 - Z_2Z_3 - Z_3Z_4$

In [60]:
# Test: class output correct Hamiltonian, J_class = '1D' by default
# # H_NN = Z1Z2 + Z2Z3 + Z3Z4

sys_1D = ising_model([0,0,0,0],[0,0,0,0])
print(np.all(sys_1D.H == sys_1D.Z.at(1) @ sys_1D.Z.at(2) + sys_1D.Z.at(2) @ sys_1D.Z.at(3) + sys_1D.Z.at(3) @ sys_1D.Z.at(4)))

True


In [63]:
X_odd_list = [sys_1D.X.at(2*i + 1) for i in range(math.ceil(sys_1D.L/2))]
X_odd = np.linalg.multi_dot(X_odd_list)
print(np.all(X_odd == sys_1D.X.at(1)@sys_1D.X.at(3)))
print(np.all(X_odd @ sys_1D.H @ X_odd ==-sys_1D.H))

True
True


### Test 2: Circular NN (Even L)
$H_{NN} = Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4 + Z_4Z_1$

$H_{odd} = X_1X_3$

$H_{odd}H_{NN}H_{odd} = -Z_1Z_2 - Z_2Z_3 - Z_3Z_4 - Z_4Z_1$

In [64]:
# Test: class output correct Hamiltonian, J_class = 'custom' and 'circular'
# # H_NN = Z1Z2 + Z2Z3 + Z3Z4 + Z4Z1

# sys = ising_model([0,0,0],[0,0,0],J_type='custom',J_custom={(1,2):1, (2,3):1, (1,3):1})
sys_circ = ising_model([0,0,0,0],[0,0,0,0],J_type='circular')
print(np.all(sys_circ.H == sys_circ.Z.at(1) @ sys_circ.Z.at(2) + sys_circ.Z.at(2) @ sys_circ.Z.at(3) + sys_circ.Z.at(3) @ sys_circ.Z.at(4) + sys_circ.Z.at(1) @ sys_circ.Z.at(4)))

True


In [67]:
print(np.all(X_odd == sys_circ.X.at(1)@sys_circ.X.at(3)))
print(np.all(X_odd @ sys_circ.H @ X_odd ==-sys_circ.H))

True
True


## Next-nearest-neighbor

### Test 1: 1D NNN
$H_{NN} = Z_1 Z_3 + Z_2 Z_4$

$H_{odd} = X_1X_3$

$H_{cor} = X_1X_2$

$H_{odd}H_{NN}H_{odd} = H_{NN}$

$H_{cor}H_{NN}H_{cor} = -H_{NN}$

In [12]:
sys_1D = ising_model([0,0,0,0],[0,0,0,0],J_type='custom', J_custom={(1,3):1, (2,4):1})
print(np.all(sys_1D.H == sys_1D.Z.at(1) @ sys_1D.Z.at(3) + sys_1D.Z.at(2) @ sys_1D.Z.at(4)))
X_odd = sys_1D.X_odd()
X_cor = sys_1D.X.at(1) @ sys_1D.X.at(2)
print(np.all(X_odd @ sys_1D.H @ X_odd == sys_1D.H))
print(np.all(X_cor @ sys_1D.H @ X_cor == -sys_1D.H))

True
True
True


In [13]:
sys_1D = ising_model([0,0,0,0],[0,0,0,0],max_coupling_range=2)
print(np.all(sys_1D.H == sys_1D.Z.at(1) @ sys_1D.Z.at(3) + sys_1D.Z.at(2) @ sys_1D.Z.at(4)))
X_odd = sys_1D.X_odd()
X_cor = sys_1D.X.at(1) @ sys_1D.X.at(2)
print(np.all(X_odd @ sys_1D.H @ X_odd == sys_1D.H))
print(np.all(X_cor @ sys_1D.H @ X_cor == -sys_1D.H))

False
False
False
